In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing, model_selection
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import random

### Data Preparation

In [2]:
def get_dataset(directory):
    data = pd.DataFrame(columns=['data', 'label'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label ='alcoholic'
                else:
                    label = 'control'

                temp_data = pd.DataFrame(columns=['data', 'label'], index=[0])

                rwb = np.load(rel_path)
                rwb.astype(np.float64)
                # with open(rel_path, 'r') as file:
                    
                #     rwb = list(csv.reader(file, delimiter=","))[0]
                #     # scaler = preprocessing.MinMaxScaler()
                #     rwb = np.asarray(rwb).astype(np.float64).reshape(-1,1)
                #     # print(rwb)
                                
                temp_data['data'][0] = rwb
                temp_data['label'] = label
                
                # decomp = np.arange(0, 366)
                # plt.plot(decomp, df_data)
                # plt.xlabel('Dimension Number')
                # plt.ylabel('Wavelet Bispectrum Energy')
                # plt.show()
                data = pd.concat([data, temp_data], ignore_index=True)
    label_map = {"alcoholic": 1, "control": 0}
    data['label_map'] = data['label'].map(label_map)      
    return data

In [3]:
def get_batch(path):
    # loading extracted feature & label
    x = get_dataset(path)

    scaler = preprocessing.MinMaxScaler()

    series_list = [
        i for i in x["data"]
    ]

    # series_list = series_list.reshape(-1, 366, 1)

    labels_list = [i for i in x["label_map"]]
        
    # y = keras.utils.to_categorical(y[0])
    dataset = tf.data.Dataset.from_tensor_slices((series_list,labels_list))
    dataset = dataset.shuffle(len(labels_list)).batch(32)

    return dataset

In [ ]:
test = get_batch('../smni_cmi_train_rwb2d_256')
for i in test:
    print(i)

### Exploratory Data Analysis

In [ ]:
# Load the CSV file into a DataFrame
signal = np.load('../Feature/Train/smni_cmi_train_feature_256\co2a0000364\co2a0000364_31_feature.npy')

# Extract the signal values from the DataFrame

# Create a time axis for the signal
t = range(len(signal))

fig, ax = plt.subplots(figsize=(10, 5))
# Plot the signal
ax.plot(t, signal)
ax.set_xlabel('Time (samples)')
ax.set_ylabel('Signal amplitude')
ax.set_title('Signal plot')
plt.show()

### Model Definition

In [4]:
def create_model():
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(61,6)))
    model.add(layers.Reshape((61, 6, 1)))

    model.add(layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=3))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

### Define Callbacks

In [5]:
def myCallbacks(log_dir):
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='acc',
    patience=50,
    mode='max')
    model_path = os.path.join(log_dir,'best_model.h5')
    mc = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    return [tensorboard_callback, early_stopping, mc]
    

### Main Program

In [6]:
lags = [256, 128, 64, 32, 16, 8, 4, 2]
folds = ['train_1', 'test_1', 'epoch_1', 'train_2', 'test_2', 'epoch_2']
time_measured = ['Wall_Time_1', 'CPU_Time_1', 'Wall_Time_2', 'CPU_Time_2']
epochs = 2000
log_dirs = ['logs1/RWB', 'logs2/RWB', 'logs3/RWB', 'logs4/RWB', 'logs5/RWB']
train_dir = '../Feature/RWB/Train/smni_cmi_train_feature'
test_dir = '../Feature/RWB/Test/smni_cmi_test_feature'



In [ ]:
for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured)
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch(train_temp_dir)
            test_ds = get_batch(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model()
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path))

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


In [6]:
train_dir = '../smni_cmi_train_rwb2d'
test_dir = '../smni_cmi_test_rwb2d'
epochs = 2000

In [9]:
fold = 2
lag = 256
log_dir = './logsSanbox'

if fold == 2:
    train_dir, test_dir = test_dir, train_dir

train_temp_dir = train_dir + '_' + str(lag)
test_temp_dir = test_dir + '_' + str(lag)

train = get_batch(train_temp_dir)
test_ds = get_batch(test_temp_dir)

train_size = int(len(list(train.as_numpy_iterator()))*0.8)
train_ds = train.take(train_size)
val_ds = train.skip(train_size)

log_path = os.path.join(log_dir, str(fold), str(lag))

model = create_model()
model.summary()

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['acc'])

cpu_start = time.process_time()
wt_start = time.time()

history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

results = model.evaluate(test_ds)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 61, 6, 1)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 4, 16)         160       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 19, 1, 16)        0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 304)               0         
                                                                 
 dense_6 (Dense)             (None, 512)               156160    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                      

In [12]:
recap.to_csv('./logs/RWB/Recap/recap.csv')
training_time.to_csv('./logs/RWB/Recap/training_time.csv')

In [1]:
! tensorboard --logdir logs --port=8000

^C


In [37]:
! cd

c:\Users\fulky\OneDrive - UNIVERSITAS INDONESIA\Documents\Akademik\Final Year\Seminar\Code\notebook
